In [1]:
import pandas as pd
import altair as alt

baseURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"

def loadData(fileName, columnName):
    data = pd.read_csv(baseURL + fileName) \
             .drop(['Lat', 'Long'], axis=1) \
             .melt(id_vars=['Province/State', 'Country/Region'], var_name='date', value_name=columnName) \
             .fillna('<all>')
    data['date'] = data['date'].astype('datetime64[ns]')
    return data

allData = loadData("time_series_19-covid-Confirmed.csv", "CumConfirmed") \
    .merge(loadData("time_series_19-covid-Deaths.csv", "CumDeaths")) \
    .merge(loadData("time_series_19-covid-Recovered.csv", "CumRecovered"))

states = ['Georgia', 'Massachusetts', 'New Jersey', 'Illinois', 'Colorado', 'Texas', 'Tennessee', 'California', 'Nebraska', 'Kansas', 'Arizona', 'Oregon', 'Utah', 'Washington']

allUsData = allData[allData['Country/Region'] == 'US']

allPKUsLocations = allUsData[allUsData['Province/State'].isin(states)]
allPKUsLocations = allPKUsLocations.rename(columns={"Province/State": "State"})
allPKUsLocations.head()

,State,Country/Region,date,CumConfirmed,CumDeaths,CumRecovered
98,Washington,US,2020-01-22,0,0,0
100,California,US,2020-01-22,0,0,0
101,Massachusetts,US,2020-01-22,0,0,0
104,Georgia,US,2020-01-22,0,0,0
105,Colorado,US,2020-01-22,0,0,0


In [2]:
states_dropdown = alt.binding_select(options=states)
genre_select = alt.selection_single(fields=['State'], bind=states_dropdown, name="PKLocation")

In [7]:
base_cum_confirmed = alt.Chart(allPKUsLocations, width=400, height=400).mark_bar().encode(x='date', y='CumConfirmed')

filter_state_cum_confirmed = base_cum_confirmed.add_selection(
    genre_select
).transform_filter(
    genre_select
).properties(title="CUMULATIVE_CONFIRMED_CASES")

base_cum_deaths = alt.Chart(allPKUsLocations, width=400, height=400).mark_bar().encode(x='date', y='CumDeaths')

filter_state_cum_death = base_cum_deaths.add_selection(
    genre_select
).transform_filter(
    genre_select
).properties(title="CUMULATIVE_DEATH_CASES")

base_cum_recovered = alt.Chart(allPKUsLocations, width=400, height=400).mark_bar().encode(x='date', y='sum(CumRecovered)')


filter_state_cum_recovered = base_cum_recovered.add_selection(
    genre_select
).transform_filter(
    genre_select
).properties(title="CUMULATIVE_RECOVERED_CASES")


# (filter_state_cum_confirmed | filter_state_cum_death | filter_state_cum_recovered)

(filter_state_cum_confirmed | filter_state_cum_death | filter_state_cum_recovered).save('chart.html')

alt.HConcatChart(...)